In [1]:
import pandas as pd
import openpyxl
import os
import re
import warnings
import xlsxwriter
import time
from bs4 import BeautifulSoup as bs
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill, Font
from random import randint
from selenium import webdriver
from selenium.webdriver.common.by import By
from string import ascii_uppercase
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

C:\Yeni klasör\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


This project has been developed for web scraping from https://www.enfsolar.com/ website. In order to use the programme, the chrome driver must be compatible with the related version of selenium package. The enf website gives a certain number of access permissions for some information. A dynamically changing ip address vpn can be used to pull data regardless of access permission. Here, only the code information is given, and the relevant VPN is left to the user's preference. When this code runs, information will be accessible to the allowed number of times without VPN. The code asks the user to enter the country number to scrape the company and product information of the relevant country. The first time the code is run, it creates an excel file that holds the scraped information. It creates a worksheet on excel file and its name is country name and saves data on this worksheet. When the code run again it creates a new worksheet which is name new country to scrape and save data in this worksheet. If the code run for same country the code updated data on the worksheet. An example excel file is added.

In [2]:
class scraping_enfsolar_seller_pages:
    
    def chrome(headless=False):
    # support to get response status and headers
        d = webdriver.DesiredCapabilities.CHROME
        d['loggingPrefs'] = {'performance': 'ALL'}
        options = webdriver.ChromeOptions()
        if headless:
            options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36")
            options.add_argument("--headless")
            options.add_argument('--no-sandbox')
            options.add_argument("--disable-setuid-sandbox")
            options.add_argument("--disable-setuid-sandbox")
            options.add_argument('--disable-dev-shm-usage')
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        options.add_argument("--disable-popup-blocking")
        chrome_deriver_manager = ChromeDriverManager()
        driver = webdriver.Chrome(chrome_deriver_manager.install(), options=options)
        driver.implicitly_wait(30)
        return driver
    
    def open_enfsolar_seller_page_to_scrape_country(self):
        driver.get('https://www.enfsolar.com/directory/seller')
        driver.implicitly_wait(3)
        parsedHTML = bs(driver.page_source, 'lxml')
        urls = []
        country = []
        counter = 0
        for h in parsedHTML.findAll('li'):
            a = h.find('a')
            try:
                if 'href' in a.attrs:
                    url = a.get('href')
                    pattern = r'^/directory/seller/[A-Za-z_%20]+'
                    match = (re.search(pattern, url))
                    urls.append(match.group(0))
                    pattern2 = r'[^a-zA-Z]+'
                    country_name = urls[counter].rsplit('/',1)[-1]
                    country_name = re.sub(pattern2, ' ', country_name)
                    country.append(country_name)
                    counter = counter + 1    
            except:
                pass
        return country, urls

    def get_page_results_info(self,parsed): 
        properties = parsed.find('table', attrs = {'class' : 'enf-list-table'}).tbody.select('td:nth-of-type(1)')
        properties2 = parsed.find('table', attrs = {'class' : 'enf-list-table'}).tbody.findAll('td', attrs = {'class' : 'no-left-right-padding'})
        c4 = parsed.find('table', attrs = {'class' : 'enf-list-table'}).tbody.findAll('td', attrs = {'class' : 'text-center'})
        company_name = []
        areas = []
        for eachproperty in range(len(properties)):
            company_name.append(properties[eachproperty].get_text().strip())
        for eachproperty in range(len(properties2)):
            areas.append(properties2[eachproperty].get_text().strip())
        liste = []
        for j in c4:
            teams = [img['alt'] for img in j('img')]
            if teams == []:
                teams = j.get_text().strip()
            j.string = ', '.join(teams)
            liste.append(teams)
            ylist = [liste[i:i + 4] for i in range(0, len(liste), 4)]
        return ylist, company_name, areas

    def dataframes(self,df4,reference_product_list,business_and_product_names,business_and_product_details):
        list_idx1 = [i for i, x in enumerate(reference_product_list) if any([j in x for j in business_and_product_names])]
        list_idx2 = [i for i, x in enumerate(business_and_product_names) if any([j in x for j in reference_product_list])] 
        final_list = ['This product no sales now']*len(reference_product_list)
        for i in range(len(list_idx1)):
            final_list[list_idx1[i]] = business_and_product_details[list_idx2[i]]
        service_coverage_language_spoken_final_list_data.append(final_list[:2])
        selector = [0,1]*len(reference_product_list[2:])
        new_final_list = final_list[2:]
        new_reference_product_list = reference_product_list[2:]
        product_details_final_list_data = [new_final_list.pop(0) if x else new_reference_product_list.pop(0) for x in selector]
        product_title_and_details_final_list_data.append(product_details_final_list_data)
          
        info1 = {'Address': company_profile_info_address,
                  'Phone' : company_profile_info_telephone,
                  'Email': company_profile_info_email,
                  'Website': company_profile_info_url,
                  'Country': company_profile_info_country,
                        }
        info1_data.append(info1)
        df1 = pd.DataFrame.from_records(info1_data)     
        column_names=['Service Coverage','Languages Spoken']
        df2 = pd.DataFrame(service_coverage_language_spoken_final_list_data, columns=column_names)
        n_repeated_titles = ['Product Title', 'Product Details']*(len(reference_product_list)-2)
        df3 = pd.DataFrame(product_title_and_details_final_list_data)
        df3.columns = n_repeated_titles       
        df5 = pd.concat([df1, df2, df3], axis=1, join='inner')  
        df6 = pd.DataFrame(ylist_f, columns = ['Distributor/Wholesaler','Brands Carried','Minimum Order Volume','Products'])
        df7 = pd.concat([df4, df6, df5], axis=1, join='inner')     
        pd.set_option('display.max_columns', None)
        pd.set_option('display.max_rows', None)
        return df7 
    
    def replace(self,business_and_product_details):  # Replacing the \n characters in the item variable with a space character
        for i, item in enumerate(business_and_product_details):
            item = re.sub(r"\s+", " ", item)
            business_and_product_details[i] = item 
        return business_and_product_details
    
    def save_csv(self,df7,country,Country_no):       
        df7['Products'] = df7['Products'].astype(str)
        df7['Products'] = df7['Products'].astype(str)
        df7['Distributor/Wholesaler'] = df7['Distributor/Wholesaler'].astype(str)
        df7['Distributor/Wholesaler'] = df7['Distributor/Wholesaler'].astype(str)  
        df7['Products'] = df7['Products'].str.replace(r"\[","")
        df7['Products'] = df7['Products'].str.replace(r"\]","")
        df7['Products'] = df7['Products'].str.replace(r'\'', '')
        df7['Distributor/Wholesaler'] = df7['Distributor/Wholesaler'].str.replace(r"\[","")
        df7['Distributor/Wholesaler'] = df7['Distributor/Wholesaler'].str.replace(r"\]","")
        df7['Distributor/Wholesaler'] = df7['Distributor/Wholesaler'].str.replace(r'\'', '')
        path = r"C:\Users\ercan\project_4_web_scraping_2\enf_seller_results.xlsx"
        if os.path.exists(path) == False:
            writer = pd.ExcelWriter("enf_seller_results.xlsx")
            df7.to_excel(writer, sheet_name=country[Country_no-1], index = False)
            print("The excel file was not exist! It was created!")
        else:
            wb = openpyxl.load_workbook('enf_seller_results.xlsx')
            writer = pd.ExcelWriter('enf_seller_results.xlsx', engine='openpyxl') 
            writer.book = wb
            df7.to_excel(writer, sheet_name=country[Country_no-1], index = False)
            print ("The excel file was exist! Added new sheet or updated!")   
        writer.save()
        writer.close()
        wb = load_workbook('enf_seller_results.xlsx')
        ws = wb[country[Country_no-1]]    
        for col in ws.columns:
            max_length = 0
            column = col[0].column_letter # Get the column name
            for cell in col:
                try: # Necessary to avoid error on empty cells
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = max_length+3
            if adjusted_width > 100:
                adjusted_width = 80    
            ws.column_dimensions[column].width = adjusted_width          
        # Creating red font and white background color for column headings
        red_font = Font(color='FF0000', name='Times New Roman', bold=True)
        white_fill = PatternFill(start_color='FFFFFF', end_color='FFFFFF', fill_type='solid')
        # Coloring all column headings in the worksheet
        for column_index in range(1, ws.max_column+1):
            column_letter = get_column_letter(column_index)
            cell = ws[f'{column_letter}1']
            cell.font = red_font
            cell.fill = white_fill
        for row in ws.rows:
            for cell in row:
        #Adjusting center alignment for the cell's text
                cell.alignment = openpyxl.styles.Alignment(horizontal='center')
        wb.save('enf_seller_results.xlsx')
        wb.close()
        return df7        

In [ ]:
if __name__ == '__main__':
    scraper = scraping_enfsolar_seller_pages()
    driver = scraper.chrome()
    country, urls = scraper.open_enfsolar_seller_page_to_scrape_country()
    for (i, item) in enumerate(country, start=1):
        print(i, item)
    Country_no = int(input('Enter no of country what you want to scrape :'))
    base_url1 = "https://www.enfsolar.com"
    base_url = base_url1 + urls[Country_no-1]
    driver.get(base_url)
    parsed_enf_solar_HTML = bs(driver.page_source, 'lxml')
    ylist, company_name, areas = scraper.get_page_results_info(parsed_enf_solar_HTML) 
    ylist_f = ylist
    company_name_f = company_name
    areas_f = areas
    tbody = bs(driver.page_source, 'lxml').find_all('td',  width='22%')
    reference_product_list = [x.get_text().strip() for x in tbody]
    reference_product_list.insert(0,'Service Coverage')
    reference_product_list.insert(1,'Languages Spoken')
    reference_product_list[4] = 'Battery'
    info1_data = []
    service_coverage_language_spoken_final_list_data = []
    product_details_final_list_data =[]
    product_title_and_details_final_list_data = []
    links_of_company = []
    for links in parsed_enf_solar_HTML.find('table', attrs = {'class' : 'enf-list-table'}).find_all('a', href=True):    
        links_of_company.append(links.get('href'))  
    if  bool(parsed_enf_solar_HTML.find('ul', class_='pagination enf-pagination')) == True :
        number_of_pages = parsed_enf_solar_HTML.find('ul', class_='pagination enf-pagination').get_text().strip()
        number_of_pages = re.findall(r'\d+', number_of_pages)
        number_of_pages = max(list(map(int, number_of_pages)))
        for page in range(2,number_of_pages+1):
            base_url_page_number = base_url1 + urls[Country_no-1] + '?page=' + '{}'.format(str(page))
            driver.get(base_url_page_number)
            parsed_enf_solar_new_page_HTML = bs(driver.page_source, 'lxml')
            try:
                links_page_new = parsed_enf_solar_new_page_HTML.find('table', attrs = {'class' : 'enf-list-table'}).find_all('a', href=True)
                ylist, company_name, areas = scraper.get_page_results_info(parsed_enf_solar_new_page_HTML)         
                ylist_f = ylist_f + ylist 
                company_name_f = company_name_f + company_name
                areas_f = areas_f + areas
                for new_links in links_page_new:
                    links_of_company.append(new_links.get('href'))
            except:
                pass 
    bar = tqdm(range(len(links_of_company)))
    bar.set_description("Number of Scraped Companies")
    for i in bar:
        driver.get(links_of_company[i])
        driver.implicitly_wait(4)
        parsed_company_info_HTML = bs(driver.page_source, 'lxml')
        try:
            phone_click = driver.find_element(By.XPATH, "//td[@class='ar:number-direction']//span[@style='cursor: pointer;']").click()
        except:
            pass
        try:
            phone_click = driver.find_element(By.XPATH, "//div[@itemprop='telephone']//span[@style='cursor: pointer;']").click()
        except:
            pass
        time.sleep(randint(1,2))
        try:
            mail_click = driver.find_element(By.XPATH, "//td[@itemprop='email']//span[@style='cursor: pointer;']").click()
        except:
            pass
        try:
            mail_click = driver.find_element(By.XPATH, "//div[@itemprop='email']//span[@style='cursor: pointer;']").click()
        except:
            pass
        if  bool(bs(driver.page_source, 'lxml').find('div', class_='enf-company-profile-info-main-spec position-relative')) == True :
            time.sleep(randint(1,2))
            script = bs(driver.page_source, 'lxml')
            try:
                company_profile_info_address = script.find('td', itemprop='address').get_text().strip()
            except:
                company_profile_info_address = 'There is no address information'
            try:
                company_profile_info_telephone = script.find('td', itemprop='telephone').get_text().strip() 
            except:
                company_profile_info_telephone = 'There is no telephone information'
            try:
                company_profile_info_email = script.find('td', itemprop='email').get_text().strip()
            except:
                company_profile_info_email = 'There is no email information'
            try:
                company_profile_info_url = script.find('a', itemprop='url').get_text().strip()
            except:
                company_profile_info_url = 'There is no url information'
            try:
                company_profile_info_country = script.find('img', class_='w-[22px] ml-px -mt-px mr-[3px] inline-block').get('title')
            except:
                company_profile_info_country = 'There is no country information'        
            business_and_product_names = [x.get_text().strip() for x in script.find('div', id='seller').findAll('div', class_='col-xs-2 enf-section-body-title')]
            business_and_product_details = [x.get_text().strip() for x in script.find('div', id='seller').findAll('div', class_='col-xs-10 enf-section-body-content blue')]    
            business_and_product_details = scraper.replace(business_and_product_details)
        else:
            new_style_script = bs(driver.page_source, 'lxml')
            try:
                company_profile_info_address = new_style_script.find('div', class_='word').get_text().strip()
            except:
                company_profile_info_address = 'There is no address information'
            try:
                company_profile_info_telephone = new_style_script.find('div', itemprop ='telephone').get_text().strip() 
            except:
                company_profile_info_telephone = 'There is no telephone information'
            try:
                company_profile_info_email = new_style_script.find('div', itemprop ='email').get_text().strip()
            except:
                company_profile_info_email = 'There is no email information'
            try:
                company_profile_info_url = new_style_script.find('a', itemprop ='url').get_text().strip()
            except:
                company_profile_info_url = 'There is no url information'
            try:
                company_profile_info_country = new_style_script.find('img', class_='w-[22px] ml-px -mt-px mr-[3px] inline-block').get('title')
            except:
                company_profile_info_country = 'There is no country information'           
            business_and_product_names = [x.get_text().strip() for x in bs(driver.page_source, 'lxml').findAll('td', class_='info-title')]
            business_and_product_details = [x.get_text().strip() for x in bs(driver.page_source, 'lxml').findAll('td', class_='info-content')]
            business_and_product_details = scraper.replace(business_and_product_details)
        df4 = pd.DataFrame()
        df4['Company Name']  = company_name_f
        df4['Areas']  = areas_f
        df7 = scraper.dataframes(df4,reference_product_list,business_and_product_names,business_and_product_details) 
    scraper.save_csv(df7,country,Country_no)   

1 Austria
2 Belgium
3 Czech Republic
4 France
5 Germany
6 Greece
7 Hungary
8 Italy
9 Netherlands
10 Poland
11 Russia
12 Spain
13 Switzerland
14 United Kingdom
15 other europe
16 Australia
17 China
18 India
19 Japan
20 Korea
21 New Zealand
22 Pakistan
23 Thailand
24 other apac
25 Brazil
26 Canada
27 Mexico
28 United States
29 other americas
30 Africa
31 Middle East
Enter no of country what you want to scrape :4


Number of Scraped Companies:  77%|████████████████████████████████████████▊            | 30/39 [08:11<02:08, 14.32s/it]